In [1]:
import requests
import pandas as pd
import numpy as np
import pebble_pb2
import datetime
import json

In [2]:
def run_query(q):
    request = requests.post('http://34.146.117.200:8000/subgraphs/name/iotex/pebble-subgraph'
                            '',
                            json={'query': q})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception('Query failed with return code: {}'.format(request.status_code))

In [3]:
query = """
{
  devices {
    id
    name
    address
    owner
    lastDataTime
    status
    name
    data
    config
  }
}
"""
result = run_query(query)

statusDf = pd.DataFrame(columns = ["Id", "Name", "Address", "Owner", "Last Data Time", "Last Data", "Status", "Raw Data",
                                  "Snr", "Vbat", "Latitude", "Longitude", "Gas Resistance", "Temperature", "Pressure",
                                  "Humidity", "Light", "Temperature2", "Gyroscope1", "Gyroscope2", "Gyroscope3", 
                                  "Accelerometer1", "Accelerometer2", "Accelerometer3"])

for key, value in result["data"].items():
    for device in value:
        data = []
        data.append(device["id"])
        data.append(device["name"])
        data.append(device["address"])
        data.append(device["owner"])
        data.append(device["lastDataTime"])
        
        if(device["lastDataTime"] == None):
            data.append("None")
        else:
            data.append(datetime.datetime.utcfromtimestamp(int(device["lastDataTime"])).strftime('%Y-%m-%d %H:%M:%S'))
            
        data.append(device["status"])
        
        if(device["data"]):
            dataString = device["data"].replace("0x", "")
            data.append(dataString)
            dataBytes = bytes.fromhex(dataString)
            sensorData = pebble_pb2.SensorData()
            sensorData.ParseFromString(dataBytes)
            
            data.append(sensorData.snr)
            data.append(sensorData.vbat)
            data.append(sensorData.latitude)
            data.append(sensorData.longitude)
            data.append(sensorData.gasResistance)
            data.append(sensorData.temperature)
            data.append(sensorData.pressure)
            data.append(sensorData.humidity)
            data.append(sensorData.light)
            data.append(sensorData.temperature2)
            if(len(sensorData.gyroscope) == 3):
                data.append(sensorData.gyroscope[0])
                data.append(sensorData.gyroscope[1])
                data.append(sensorData.gyroscope[2])
            else:
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
            if(len(sensorData.accelerometer) == 3):
                data.append(sensorData.accelerometer[0])
                data.append(sensorData.accelerometer[1])
                data.append(sensorData.accelerometer[2])
            else:
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
        else:
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            data.append("No Data")
            
        statusDf.loc[len(statusDf)] = data

In [4]:
statusDf.sort_values(by='Last Data Time', ascending = False, inplace=True)
statusDf.reset_index(inplace=True)

In [5]:
timeDf = statusDf[statusDf["Last Data"] != "None"]

In [6]:
timeDf['Latitude'] = timeDf['Latitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
timeDf['Longitude'] = timeDf['Longitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))

timeDf["Longitude"] = pd.to_numeric(timeDf["Longitude"])
timeDf["Latitude"] = pd.to_numeric(timeDf["Latitude"])

<ipython-input-6-c2acca8b7cd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timeDf['Latitude'] = timeDf['Latitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
<ipython-input-6-c2acca8b7cd5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timeDf['Longitude'] = timeDf['Longitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
<ipython-input-6-c2acca8b7cd5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

# Device

In [74]:
def getDeviceData(imei):
    query = """
    {
      deviceRecords(where: { imei: "%s" }) {
        raw
        imei
        signature
        timestamp
      }
    }
    """ %(imei)

    result = run_query(query)

    deviceDf = pd.DataFrame(columns = ["Id", "Timestamp", "Raw Data", "Snr", "Vbat", "Latitude", "Longitude", "Gas Resistance", "Temperature", 
                                       "Pressure", "Humidity", "Light", "Temperature2", "Gyroscope1", "Gyroscope2", "Gyroscope3", 
                                      "Accelerometer1", "Accelerometer2", "Accelerometer3"])

    for key, value in result["data"].items():
        for subkey in value:
            data = []
            data.append(imei)

            if((subkey['raw'] != "0x") and (subkey['raw'])):
                dataString = subkey['raw'].replace("0x", "")
                data.append(datetime.datetime.utcfromtimestamp(int(subkey["timestamp"])).strftime('%Y-%m-%d %H:%M:%S'))
                data.append(dataString)
                dataBytes = bytes.fromhex(dataString)
                sensorData = pebble_pb2.SensorData()

                sensorData.ParseFromString(dataBytes)
                
                data.append(sensorData.snr)
                data.append(sensorData.vbat)
                data.append(sensorData.latitude)
                data.append(sensorData.longitude)
                data.append(sensorData.gasResistance)
                data.append(sensorData.temperature)
                data.append(sensorData.pressure)
                data.append(sensorData.humidity)
                data.append(sensorData.light)
                data.append(sensorData.temperature2)
                if(len(sensorData.gyroscope) == 3):
                    data.append(sensorData.gyroscope[0])
                    data.append(sensorData.gyroscope[1])
                    data.append(sensorData.gyroscope[2])
                else:
                    data.append("No Data")
                    data.append("No Data")
                    data.append("No Data")
                if(len(sensorData.accelerometer) == 3):
                    data.append(sensorData.accelerometer[0])
                    data.append(sensorData.accelerometer[1])
                    data.append(sensorData.accelerometer[2])
                else:
                    data.append("No Data")
                    data.append("No Data")
                    data.append("No Data")
            else:
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")
                data.append("No Data")

            deviceDf.loc[len(deviceDf)] = data

    deviceDf['Latitude'] = deviceDf['Latitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
    deviceDf['Longitude'] = deviceDf['Longitude'].apply(lambda x: x / 10 ** (len((str(x))) - 2))

    deviceDf["Longitude"] = pd.to_numeric(deviceDf["Longitude"])
    deviceDf["Latitude"] = pd.to_numeric(deviceDf["Latitude"])
    
    deviceDf.sort_values(by='Timestamp', ascending = False, inplace=True)
    deviceDf.reset_index(inplace=True)

    return deviceDf

In [75]:
deviceDf = getDeviceData("100000000000005")

In [76]:
deviceDf

,index,Id,Timestamp,Raw Data,Snr,Vbat,Latitude,Longitude,Gas Resistance,Temperature,Pressure,Humidity,Light,Temperature2,Gyroscope1,Gyroscope2,Gyroscope3,Accelerometer1,Accelerometer2,Accelerometer3
0,79,100000000000005,2021-09-15 17:16:05,082710201828200e2826301e38164026481f5025583c58...,39,32,40.0,14.0,38,30,22,38,31,37,30,25,29,28,37,29
1,27,100000000000005,2021-09-15 17:16:04,082710201828200e2825301f381540194822501d582e58...,39,32,40.0,14.0,37,31,21,25,34,29,23,30,30,36,31,30
2,81,100000000000005,2021-09-15 17:15:55,081710271828200e2816301b3816402248165020583c58...,23,39,40.0,14.0,22,27,22,34,22,32,30,32,32,29,23,37
3,50,100000000000005,2021-09-15 17:15:34,082010211828200e281a3020381c402348225024582c58...,32,33,40.0,14.0,26,32,28,35,34,36,22,36,32,21,36,27
4,87,100000000000005,2021-09-15 17:15:33,082610151828200e281b301a3822401948165025584a58...,38,21,40.0,14.0,27,26,34,25,22,37,37,35,35,33,26,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,44,100000000000005,2021-09-15 17:03:38,081510171829200e281f301c381b402048255025584458...,21,23,41.0,14.0,31,28,27,32,37,37,34,23,37,32,28,28
96,54,100000000000005,2021-09-15 17:03:32,081910271829200e281b302138234020481b501e584258...,25,39,41.0,14.0,27,33,35,32,27,30,33,34,39,36,28,23
97,84,100000000000005,2021-09-15 17:03:24,082610211829200e2822301938244015481d501b582e58...,38,33,41.0,14.0,34,25,36,21,29,27,23,37,39,26,31,24
98,14,100000000000005,2021-09-15 17:03:16,081e10181829200e281f3017381d40234823501a584e58...,30,24,41.0,14.0,31,23,29,35,35,26,39,25,38,25,34,27
